# import

In [1]:
import cv2
import os
import os.path as osp
from glob import glob
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# path

In [2]:
data_name = 'player'
input_dir = osp.join('../input', data_name)
data_dir = osp.join('../output', data_name, 'tracking')
label_save_dir = osp.join('../output', data_name, 'label')
reid_dir = osp.join('../output', data_name, 'reid')
track_label_path = osp.join('../output/')
movie_dir = osp.join('../output', data_name, 'movie')
mot_pid_dir = osp.join('../output', data_name, 'mot_pid')

# function

In [3]:
def listdirs(dir_path, name_condition=None):
    dir_names = os.listdir(dir_path)
    if name_condition is not None:
        dir_names = [dir_name for dir_name in dir_names if dir_name.startswith(name_condition)]
    dir_names.sort()
    return dir_names

In [4]:
def save_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)


def load_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

In [5]:
def draw_bbox(img, bboxes, c=(0,255,0)):
    for bbox in bboxes:
        cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), 
                      (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), 
                      (0,255,0), 2, lineType=cv2.LINE_AA)
        ct = [bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2]
        txt = '{}'.format(bbox[4])
        cv2.putText(img, txt, (int(ct[0]), int(ct[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    (0, 0, 0), thickness=3, lineType=cv2.LINE_AA)
        cv2.putText(img, txt, (int(ct[0]), int(ct[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    c, thickness=2, lineType=cv2.LINE_AA)

# track pid

In [6]:
game = 'game1'

#  mot_pid

In [11]:
games = listdirs(input_dir, 'game')
columns = ['frame_index', 'tracking_id', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'image_name']
for game in games:
    game_dir = osp.join(input_dir, game)
    clips = listdirs(game_dir, 'Clip')
    
    track_pid_paths = glob(osp.join(reid_dir, game, '*.csv'))
    track_pid_df_list = []
    
    for track_pid_path in track_pid_paths:
        person_name = osp.splitext(osp.basename(track_pid_path))[0]
        print(person_name)
        track_pid_df = pd.read_csv(track_pid_path)
        
        track_pid_df = track_pid_df[track_pid_df.person==1]
        track_pid_df['person_name'] = person_name
        
        track_pid_df_list.append(track_pid_df)
    track_pid_df = pd.concat(track_pid_df_list)
    
    for clip in clips:
        track_pid_clip_df = track_pid_df[track_pid_df['clip']==clip]
        
        clip_dir = osp.join(game_dir, clip)
        file_names = listdirs(clip_dir)
        image_names = [file_name for file_name in file_names if osp.splitext(file_name)[1]=='.jpg']
        
        label_path = osp.join(label_save_dir, game, '{}.csv'.format(clip))
        label_df = pd.read_csv(label_path)
        
        track_pid_clip_df = track_pid_clip_df[['tracking_id', 'person_name']]
        mot_pid_df = label_df.merge(track_pid_clip_df, on=['tracking_id'], how='left')
        mot_pid_df_save_path = osp.join(mot_pid_dir, game, '{}.csv'.format(clip))
        
        
        os.makedirs(osp.dirname(mot_pid_df_save_path), exist_ok=True)
        mot_pid_df.to_csv(mot_pid_df_save_path, index=False)

Federer
Djokovic
Unknown_Blue
Unknown_Skyblue
Djokovic
Monfils
Murray
Djokovic
Unknown_WhiteBlack
Unknown_WhiteWhite
Unknown_WhiteBlue
Unknown_Skyblue
Ferrer
Fognini
Federer
Kyrgios
Unknown_Orange
Unknown_Purple
Sharapova
Halep


In [10]:
mot_pid_df

,tracking_id,frame_index,xmin,ymin,xmax,ymax,confidence,image_name,width,height,person_name
0,1,1,594,90,620,158,0.656887,0001.jpg,26,68,Halep
1,1,2,595,90,619,155,0.554561,0002.jpg,24,65,Halep
2,1,3,594,91,619,156,0.597442,0003.jpg,25,65,Halep
3,1,4,593,95,619,153,0.550981,0004.jpg,26,58,Halep
4,1,5,593,98,617,150,0.537664,0005.jpg,24,52,Halep
...,...,...,...,...,...,...,...,...,...,...,...
1902,37,78,1257,213,1271,256,0.286542,0078.jpg,14,43,NaN
1903,37,79,1257,213,1272,256,0.281872,0079.jpg,15,43,NaN
1904,37,80,1257,212,1272,257,0.288108,0080.jpg,15,45,NaN
1905,37,81,1257,215,1272,255,0.317298,0081.jpg,15,40,NaN


# movie_check

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'H264')
columns = ['frame_index', 'tracking_id', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'image_name']

In [32]:
game = 'game1'
clip = 'Clip5'

In [ ]:
from IPython.display import Video

In [33]:
game_dir = osp.join(input_dir, game)

track_pid_paths = glob(osp.join(reid_dir, game, '*.csv'))
track_pid_df_list = []

for track_pid_path in track_pid_paths:
    person_name = osp.splitext(osp.basename(track_pid_path))[0]
    print(person_name)
    track_pid_df = pd.read_csv(track_pid_path)

    track_pid_df = track_pid_df[track_pid_df.person==1]
    track_pid_df['person_name'] = person_name

    track_pid_df_list.append(track_pid_df)
track_pid_df = pd.concat(track_pid_df_list)

clip_dir = osp.join(game_dir, clip)
file_names = listdirs(clip_dir)
image_names = [file_name for file_name in file_names if osp.splitext(file_name)[1]=='.jpg']

label_path = osp.join(label_save_dir, game, '{}.csv'.format(clip))
label_df = pd.read_csv(label_path)
results = label_df[['frame_index', 'xmin', 'ymin', 'width', 'height', 'tracking_id']].values

movie_save_path = osp.join(movie_dir, game, '{}.mp4'.format(clip))
os.makedirs(osp.dirname(movie_save_path), exist_ok=True)
out = cv2.VideoWriter(movie_save_path, fourcc, 30, (1280, 720))

print(game, clip)
for image_name in image_names:
    frame_index = osp.splitext(image_name)[0]
    frame_path = osp.join(clip_dir, image_name)

    frame = cv2.imread(frame_path)
    bboxes = results[results[:, 0]==int(frame_index)][:, [1,2,3,4,5]]

    for bbox in bboxes:
        ct = [bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2]

        df = track_pid_df[track_pid_df.tracking_id==bbox[4]]
        df = df[df['clip']==clip]
        if len(df) > 0:
            c = (0, 0, 255)
            person_name = df['person_name'].values[0]
            cv2.putText(frame, person_name, (int(ct[0]) + 20, int(ct[1]) + 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    (0, 0, 0), thickness=3, lineType=cv2.LINE_AA)
            cv2.putText(frame, person_name, (int(ct[0]) + 20, int(ct[1]) + 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    c, thickness=2, lineType=cv2.LINE_AA)
        else:
            c = (0, 255, 0)

        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                      (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), 
                      c, 2, lineType=cv2.LINE_AA)

        txt = '{}'.format(bbox[4])

        cv2.putText(frame, txt, (int(ct[0]), int(ct[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    (0, 0, 0), thickness=3, lineType=cv2.LINE_AA)
        cv2.putText(frame, txt, (int(ct[0]), int(ct[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    c, thickness=2, lineType=cv2.LINE_AA)

    cv2.putText(frame, str(frame_index), (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0, 0, 0), thickness=3, lineType=cv2.LINE_AA)
    cv2.putText(frame, str(frame_index), (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)


    # cv2.imshow('image',frame)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
out.release()

Federer
Djokovic
game1 Clip5


In [34]:
Video(movie_save_path, width=640)